In [10]:
import pandas as pd
pd.options.display.max_columns = None
import re
import os
import time
from tqdm import tqdm

for dirname, _, filenames in os.walk('/home/damar.pramuditya/School/Data Science/unstructure_assignment/input/archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/damar.pramuditya/School/Data Science/unstructure_assignment/input/archive/client_hostname.csv
/home/damar.pramuditya/School/Data Science/unstructure_assignment/input/archive/access.log


In [7]:
!ls -lsSh 'input/archive/access.log'

3,3G -rw-rw-r-- 1 damar.pramuditya damar.pramuditya 3,3G Feb 13  2021 input/archive/access.log


In [9]:
!head -n 4 'input/archive/access.log'

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"
31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"
31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"
40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/1.1" 200 1696 

In [11]:
combined_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

In [12]:
import time
import re
import pandas as pd

def logs_to_df(logfile, output_dir, errors_file):
    with open(logfile) as source_file:
        linenumber = 0
        parsed_lines = []
        for line in tqdm(source_file):
            try:
                log_line = re.findall(combined_regex, line)[0]
                parsed_lines.append(log_line)
            except Exception as e:
                with open(errors_file, 'at') as errfile:
                    print((line, str(e)), file=errfile)
                continue
            linenumber += 1
            if linenumber % 250_000 == 0:
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
                parsed_lines.clear()
        else:
            df = pd.DataFrame(parsed_lines, columns=columns)
            df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
            parsed_lines.clear()

In [14]:
%%capture
!pip install advertools

In [15]:
%time logs_to_df(logfile='input/archive/access.log', output_dir='output/', errors_file='errors.txt')

10365152it [00:52, 199296.97it/s]


CPU times: user 48.2 s, sys: 4.47 s, total: 52.7 s
Wall time: 52.3 s


In [18]:
# check errors
!wc errors.txt

   292   7151 229770 errors.txt


In [19]:
%time logs_df = pd.read_parquet('output/')

CPU times: user 7.7 s, sys: 2.14 s, total: 9.84 s
Wall time: 4.92 s


In [20]:
!du -sh output/

257M	output/


In [23]:
val = 257/3300
print(val)

0.07787878787878788


In [24]:
logs_df.info(show_counts=True, verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364865 entries, 0 to 10364864
Data columns (total 9 columns):
 #   Column      Non-Null Count     Dtype 
---  ------      --------------     ----- 
 0   client      10364865 non-null  object
 1   userid      10364865 non-null  object
 2   datetime    10364865 non-null  object
 3   method      10364865 non-null  object
 4   request     10364865 non-null  object
 5   status      10364865 non-null  object
 6   size        10364865 non-null  object
 7   referer     10364865 non-null  object
 8   user_agent  10364865 non-null  object
dtypes: object(9)
memory usage: 711.7+ MB


In [25]:
# 711 MB. We now remove the files in parquet_dir and optimize the datatypes and use more efficient ones.
%rm -r output/

In [26]:
logs_df['client'] = logs_df['client'].astype('category')
del logs_df['userid']
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], format='%d/%b/%Y:%H:%M:%S %z')
logs_df['method'] = logs_df['method'].astype('category')
logs_df['status'] = logs_df['status'].astype('int16')
logs_df['size'] = logs_df['size'].astype('int32')
logs_df['referer'] = logs_df['referer'].astype('category')
logs_df['user_agent'] = logs_df['user_agent'].astype('category')

In [27]:
logs_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364865 entries, 0 to 10364864
Data columns (total 8 columns):
 #   Column      Non-Null Count     Dtype                                
---  ------      --------------     -----                                
 0   client      10364865 non-null  category                             
 1   datetime    10364865 non-null  datetime64[ns, pytz.FixedOffset(210)]
 2   method      10364865 non-null  category                             
 3   request     10364865 non-null  object                               
 4   status      10364865 non-null  int16                                
 5   size        10364865 non-null  int32                                
 6   referer     10364865 non-null  category                             
 7   user_agent  10364865 non-null  category                             
dtypes: category(4), datetime64[ns, pytz.FixedOffset(210)](1), int16(1), int32(1), object(1)
memory usage: 342.3+ MB


In [28]:
# The file was reduced further from 711 to 342 MB. (342 ÷ 711 = 0.48 of the original size)

# We now save it to a single file, and read again.

In [29]:
%time logs_df.to_parquet('logs_df.parquet')

CPU times: user 3.78 s, sys: 1.01 s, total: 4.79 s
Wall time: 4.44 s


In [30]:
%time logs_df = pd.read_parquet('logs_df.parquet')

CPU times: user 3.43 s, sys: 1.66 s, total: 5.09 s
Wall time: 3.56 s


In [31]:
logs_df.shape

(10364865, 8)

In [32]:
logs_df

,client,datetime,method,request,status,size,referer,user_agent
0,37.152.163.59,2019-01-22 12:38:27+03:30,GET,/image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-7.j...,200,1105,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
1,37.152.163.59,2019-01-22 12:38:27+03:30,GET,/static/images/zanbil-kharid.png,200,358,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
2,85.9.73.119,2019-01-22 12:38:27+03:30,GET,/static/images/next.png,200,3045,https://znbl.ir/static/bundle-bundle_site_head...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
3,37.152.163.59,2019-01-22 12:38:27+03:30,GET,/image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-4.j...,200,1457,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
4,85.9.73.119,2019-01-22 12:38:27+03:30,GET,/static/images/checked.png,200,1083,https://znbl.ir/static/bundle-bundle_site_head...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
...,...,...,...,...,...,...,...,...
10364860,86.104.110.254,2019-01-26 16:01:31+03:30,GET,/settings/logo,200,4120,https://www.zanbil.ir/m/browse/tv/%D8%AA%D9%84...,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like M...
10364861,5.125.254.169,2019-01-26 16:01:31+03:30,GET,/image/5/brand,200,2171,https://www.zanbil.ir/m/filter/p62%2Cstexists,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_2 like...
10364862,65.49.68.192,2019-01-26 16:01:31+03:30,GET,/image/64646/productModel/150x150,200,5318,https://www.zanbil.ir/browse/audio-and-video-e...,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...
10364863,5.125.254.169,2019-01-26 16:01:31+03:30,GET,/image/1/brand,200,3924,https://www.zanbil.ir/m/filter/p62%2Cstexists,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_2 like...
